In [6]:
import logging
logging.root.handlers = []  # Jupyter messes up logging so needs a reset
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
import pandas as pd
import numpy as np
import gensim
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.corpus import stopwords
from pprint import pprint
from sklearn.datasets import fetch_20newsgroups

In [37]:
newsgroups_train_meta = fetch_20newsgroups(subset='train')

## Categorias dos textos

In [8]:
pprint(list(newsgroups_train_meta.target_names))

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']


### Vamos ver um texto de Motos

In [39]:
newsgroups_train_meta.target_names[newsgroups_train_meta.target[10]]

'rec.motorcycles'

In [40]:
newsgroups_train_meta.data[10]

u'From: irwin@cmptrc.lonestar.org (Irwin Arnstein)\nSubject: Re: Recommendation on Duc\nSummary: What\'s it worth?\nDistribution: usa\nExpires: Sat, 1 May 1993 05:00:00 GMT\nOrganization: CompuTrac Inc., Richardson TX\nKeywords: Ducati, GTS, How much? \nLines: 13\n\nI have a line on a Ducati 900GTS 1978 model with 17k on the clock.  Runs\nvery well, paint is the bronze/brown/orange faded out, leaks a bit of oil\nand pops out of 1st with hard accel.  The shop will fix trans and oil \nleak.  They sold the bike to the 1 and only owner.  They want $3495, and\nI am thinking more like $3K.  Any opinions out there?  Please email me.\nThanks.  It would be a nice stable mate to the Beemer.  Then I\'ll get\na jap bike and call myself Axis Motors!\n\n-- \n-----------------------------------------------------------------------\n"Tuba" (Irwin)      "I honk therefore I am"     CompuTrac-Richardson,Tx\nirwin@cmptrc.lonestar.org    DoD #0826          (R75/6)\n------------------------------------------

### Agora um de computação

In [41]:
newsgroups_train_meta.target_names[newsgroups_train_meta.target[1]]

'comp.sys.mac.hardware'

In [42]:
newsgroups_train_meta.data[1]

u"From: guykuo@carson.u.washington.edu (Guy Kuo)\nSubject: SI Clock Poll - Final Call\nSummary: Final call for SI clock reports\nKeywords: SI,acceleration,clock,upgrade\nArticle-I.D.: shelley.1qvfo9INNc3s\nOrganization: University of Washington\nLines: 11\nNNTP-Posting-Host: carson.u.washington.edu\n\nA fair number of brave souls who upgraded their SI clock oscillator have\nshared their experiences for this poll. Please send a brief message detailing\nyour experiences with the procedure. Top speed attained, CPU rated speed,\nadd on cards and adapters, heat sinks, hour of usage per day, floppy disk\nfunctionality with 800 and 1.4 m floppies are especially requested.\n\nI will be summarizing in the next two days, so please add to the network\nknowledge base if you have done the clock upgrade and haven't answered this\npoll. Thanks.\n\nGuy Kuo <guykuo@u.washington.edu>\n"

### Remover os headers, footers e quotes dos textos (Metadados que não vão nos interessar hoje)

In [43]:
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))

### Mesmo texto de computação

In [45]:
newsgroups_train.data[1]

u"A fair number of brave souls who upgraded their SI clock oscillator have\nshared their experiences for this poll. Please send a brief message detailing\nyour experiences with the procedure. Top speed attained, CPU rated speed,\nadd on cards and adapters, heat sinks, hour of usage per day, floppy disk\nfunctionality with 800 and 1.4 m floppies are especially requested.\n\nI will be summarizing in the next two days, so please add to the network\nknowledge base if you have done the clock upgrade and haven't answered this\npoll. Thanks."

Bem melhor o/

### Tokenização + Limpeza + Stopwords

Atenção! Cada problema que envolve texto exige um tratamento diferente nesta etapa. Ex. Caso o seu problema envolva análise de sentimento pode não ser uma boa ideia remover pontuação.

O pacote nltk tem bastante coisa de NLP que iremos aproveitar - tem algumas coisas para o português também!

In [46]:
def clean_and_tokenizing(text):
    # Remove numbers + punctuation        
    letters_only = re.sub("[^a-zA-Z]", " ", text) 

    # Convert to lower case, split into individual words
    words = letters_only.lower().split()                             

    # Save english stopwords
    stops = set(stopwords.words("english"))                  

    # Remove stop words
    meaningful_words = [w for w in words if not w in stops]   
    
    # Join everython again
    return( " ".join( meaningful_words ))   

In [ ]:
print "Cleaning and parsing the training set movie reviews...\n"
clean_texts = []
for i in xrange(0, num_text):
    # If the index is evenly divisible by 1000, print a message
    if( (i+1)%1000 == 0 ):
        print "Review %d of %d\n" % ( i+1, num_text )
    clean_texts.append(review_to_words(newsgroups_train["data"][i]))